In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [5]:
df = pd.read_csv("train.csv", index_col="Id")

In [3]:
df['LotFrontage'] = df.loc[df['LotFrontage']<160]['LotFrontage']
df['LotFrontage'] = df.groupby('YearBuilt')['LotFrontage'].transform(lambda x: x.fillna(x.mean()))

LotShape_dict = {'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3}
df['LotShape'] = [LotShape_dict[l] for l in df['LotShape']]
df['LotArea'] = df.loc[df['LotArea']<60000]['LotArea']

df['Alley'] = df['Alley'].fillna("NA")

# condition1 and condition2 have the same values, nominal should be only one
encoder = OneHotEncoder() # Initialize the OneHotEncoder

encoded_cond1 = encoder.fit_transform(df[['Condition1']])
original_cond1_names = encoder.get_feature_names_out(['Condition1'])
encoded_cond1_df = pd.DataFrame(encoded_cond1.toarray(), columns=original_cond1_names)

encoded_cond2 = encoder.fit_transform(df[['Condition2']])
original_cond2_names = encoder.get_feature_names_out(['Condition2'])
encoded_cond2_df = pd.DataFrame(encoded_cond2.toarray(), columns=original_cond2_names)

def df_union(df1, df2, prefix2, on=1):
    combined_df = df1.copy()
    
    for i, row in df1.iterrows():
        for col_name, _ in row.items():
            df2_col_name = prefix2+"_"+col_name.split("_")[1]
            if df2_col_name in df2.columns.values:
                if df2.loc[i][df2_col_name] == 1:
                    combined_df.loc[i][col_name] = 1
        
    return combined_df


encoded_conditions_df = df_union(encoded_cond1_df, encoded_cond2_df, "Condition2")
df = df.drop(["Condition1", 'Condition2'], axis=1)
df = df.append(encoded_conditions_df, ignore_index=True)

df['HouseAge'] = df['YrSold'] - df['YearBuilt']
df = df.drop('YrSold', axis=1)
df = df.drop('MoSold', axis=1)
df = df.drop('YearBuilt', axis=1)

df['isRemodelled'] = df.apply(lambda x: 0 if x['YearRemodAdd'] == x['YearBuilt'] else 1, axis=1)
df = df.drop('YearRemodAdd', axis=1)

 

df = df.dropna(subset=['MasVnrType'])

df['Fence'] = df['Fence'].fillna("NA")

In [ ]:
encoder = OneHotEncoder(sparse_output=False) # Initialize the OneHotEncoder
encoded_data = encoder.fit_transform(df[['MSSubClass', 'MSZoning', 'BldgType', 'HouseStyle', 'Street', 'Alley', 'Neighborhood', 'Condition1', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]) # Fit and transform the data
column_names = encoder.get_feature_names_out(['GarageType']) # Create column names for the encoded variables
df[column_names] = encoded_data # Concatenate the encoded columns to the original DataFrame